# **FORWARD DIFFUSION**
**License : CC BY-NC-SA 4.0**

In this notebook, we demonstrate how to apply the forward diffusion on Fashion MNIST.  
Source :  
- https://huggingface.co/blog/annotated-diffusion  
- https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/annotated_diffusion.ipynb#scrollTo=5153024b  

## **Jean Zay Environment**
We should use the `pytorch-gpu-2.1.1_py3.11.5` jupyter kernel.

In [ ]:
!module list

## **Import libraries**

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np
from datasets import load_dataset
from torchvision.utils import make_grid

from code.fashion_mnist_dataset import get_dataloader
from code.forward_diffusion import get_alph_bet
from code.utils import show_images

## **Dataset loading**
We will get Fashion MNIST (https://huggingface.co/datasets/fashion_mnist) by using the library `datasets` (https://huggingface.co/docs/datasets/index).

In [ ]:
dataset = load_dataset("fashion_mnist")

In [ ]:
# we inspect the dataset
print(dataset)

Our `dataset` object is composed of two subpart: `train` and `test`.  
`train` contains 60000 samples of `image` and `label` :
- `image` is a PIL object of 28x28 grayscale image
- `label` gives the class of the object among 10 availables **(Pullover, Sneaker, etc.)**

In [ ]:
# get `train` 
train_dataset = dataset['train']
# let's see the first sample of `train`
print(train_dataset[0])

In [ ]:
# display the image in the first sample
display(train_dataset[0]['image'])

## **Dataloader creation**

`get_dataloader` will load the Fashion MNIST dataset, apply all the transformation we want on it and encapsulate that dataset in a dataloader.

In [ ]:
# ARGUMENTS
batch_size = 64

# GET DATALOADER
train_dataloader, channels, image_size, n_batch = get_dataloader(batch_size=batch_size,
                                                                 test=False)

# GET A BATCH
batch_image = next(iter(train_dataloader))['pixel_values']

# PRINT TYPE AND SHAPE of 1 BATCH
print(f"\nBatch ==>\n",
      f"Type      {type(batch_image)}\n",
      f"Shape     {batch_image.shape}")

## **Visualization**

`show_images` will display images from a batch of tensors

In [ ]:
show_images(next(iter(train_dataloader))['pixel_values'])

## **Apply forward diffusion**
The aim of this part is to create a function that will noise any image at any step (following the DDPM diffusion process).

### **Beta scheduling**
First, we create a function that will compute every betas of every steps (following a specific shedule). The linear schedule is already given (original DDPM).  
**TODO : write the cosine schedule (improved DDPM)**  
![Cosine](img/cosine_schedule.png)

In [ ]:
def linear_beta_schedule(timesteps, beta_start = 0.0001, beta_end = 0.02):
    """
    linar schedule from the original DDPM paper https://arxiv.org/abs/2006.11239
    """
    return torch.linspace(beta_start, beta_end, timesteps)


def cosine_beta_schedule(timesteps, s=0.008):
    """
    cosine schedule as proposed in https://arxiv.org/abs/2102.09672
    Solution in `code.forward_diffusion.py`
    """
    steps = timesteps + 1
    x = torch.linspace(0, timesteps, steps)
    alphas_cumprod = None
    betas = None
    return torch.clip(betas, 0.0001, 0.9999)

In [ ]:
# CREATE LINEAR SCHEDULE
linear_sched = linear_beta_schedule(1000)

# PRINT TYPE AND SHAPE
print(f"\nlinear_sched ==>\n",
      f"Type      {type(linear_sched)}\n",
      f"Shape     {linear_sched.shape}")

### **Constants calculation**
`get_alph_bet` calculates every constants we need for our Diffusion Model. <br/><br/>
**Reminder :**
- $ T $ : **ARGUMENT for get_alph_bet / Total number of noising steps**
- $ \beta_t $ : **ARGUMENT for get_alph_bet / Level of noise applied at each timestep t**  
- $ \alpha_t = 1 - \beta_t $ : **alphas**  
- $ \bar{\alpha}_t = \prod_{k=1}^t\alpha_k $ : **alphas_cumprod**

**Constants:**
- $ \beta_t $ : **betas**
- $ \sqrt{\frac{1}{\alpha_t}} $ : **sqrt_recip_alphas**
- $ \sqrt{\bar{\alpha}_t} $: **sqrt_alphas_cumprod**
- $ \sqrt{1-\bar{\alpha}_t} $: **sqrt_one_minus_alphas_cumprod**
- $ \tilde{\beta}_t = \beta_t\frac{1-\bar{\alpha}_{t-1}}{1-\bar{\alpha}_t} $: **posterior_variance**

In [ ]:
# ARGUMENTS
T = 1000

# CREATE BETA SCHEDULERS
const_linear_dict = get_alph_bet(T, schedule=linear_beta_schedule)
const_cosine_dict = get_alph_bet(T, schedule=cosine_beta_schedule)

# PRINT KEYS
print(const_linear_dict.keys())

In [ ]:
# Create a figure and two subplots horizontally
fig, axs = plt.subplots(1, 2, figsize=(10, 5))  # 1 row, 2 columns

# Plot the first curve on the first subplot
axs[0].plot(np.arange(T), const_linear_dict['betas'])
axs[0].set_title('Level of noise at time t ')  # Set title for the first subplot
axs[0].set_xlabel('Step')
axs[0].set_ylabel('q(x_t|x_t-1)')

# Plot the second curve on the second subplot
axs[1].plot(np.arange(T), const_linear_dict['sqrt_one_minus_alphas_cumprod'])
axs[1].set_title('Level of noise added to the original image')  # Set title for the second subplot
axs[1].set_xlabel('Step')
axs[1].set_ylabel('q(x_t|x_0)')

# Adjust layout to prevent overlap
plt.tight_layout()

# Display the plot
plt.show()

### **Computation of $ q(x_t|x_0) $**

#### **extract function**

- **constants** = const_cosine_dict['betas'], const_linear_dict['posterior_variance'], etc.
- **batch_t** = a batch of steps for the forward diffusion (for each data in our batch, we determine the step of which we will apply the forward diffusion)
- **x_shape** = shape of our batch of data

In [ ]:
def extract(constants, batch_t, x_shape):
    '''Extract the values needed for time t'''
    
    diffusion_batch_size = batch_t.shape[0]
    
    # get a list of the appropriate constants of each timesteps
    out = constants.gather(-1, batch_t.cpu()) 
    
    return out.reshape(diffusion_batch_size, *((1,) * (len(x_shape) - 1))).to(batch_t.device)

**Question :**
- What is the value of x_shape ?
- What is the shape of batch_t compared to x_shape ?
- What is the shape of the output ?
- Can we multiply the output of extract with our image ?

In [ ]:
# we define x_shape according to `batch_image`
x_shape = batch_image.shape
print(f"x_shape = {x_shape}")

In [ ]:
# we define batch_t according to x_shape
batch_t = torch.randint(1, T+1, (x_shape[0],)) # random int in range of [1, T+1] 
print(f"\nbatch_t ==>\n",
      f"Shape       {batch_t.shape}\n",
      f"Content     {batch_t}")

In [ ]:
# extract sqrt_alphas_cumprod_t
sqrt_alphas_cumprod_t = extract(const_linear_dict['sqrt_alphas_cumprod'], batch_t, batch_image.shape)
print(f"Shape {sqrt_alphas_cumprod_t.shape}")

In [ ]:
# TODO : multiply sqrt_alphas_cumprod_t and batch_image
multiply_output = None
print(multiply_output.shape)

#### **Computation**

Now that we have every constants that we need, we can create a function that will add noise to an image following the forward diffusion process.  
`q_sample` corresponds to $ q(x_t|x_0) $:

![q_sample](img/forward_diffusion_process.png)

In [ ]:
def q_sample(constants_dict, batch_x0, batch_t, noise=None):
    """
    Solution in `code.forward_diffusion.py`
    """
    if noise is None:
        noise = torch.randn_like(batch_x0)
    
    # TODO : 
    # Define the noisy image for each image in batch_x0 associated with a level of noise in batch_t
    
    sqrt_alphas_cumprod_t = None
    sqrt_one_minus_alphas_cumprod_t = None
    
    noisy_batch_x0 = batch_x0
    
    return noisy_batch_x0

We can now visualize how the forward diffusion process adds noise gradually the image according to its parameters:

In [ ]:
# ARGUMENTS
T = 1000
const_linear_dict = get_alph_bet(T, schedule=linear_beta_schedule)
const_cosine_dict = get_alph_bet(T, schedule=cosine_beta_schedule)
batch_t = torch.arange(batch_size)*(T//batch_size)  # get a range of timesteps from 0 to T


print(f"timesteps: {batch_t}")
noisy_batch_linear = q_sample(const_linear_dict, batch_image, batch_t, noise=None)
noisy_batch_cosine = q_sample(const_cosine_dict, batch_image, batch_t, noise=None)

print("Original images:")
show_images(batch_image[:])

print("Noised images with linear shedule:")
show_images(noisy_batch_linear[:])

print("Noised images with cosine shedule:")
show_images(noisy_batch_cosine[:])